In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
from mpl_toolkits.basemap import Basemap
from matplotlib.colors import LinearSegmentedColormap, Normalize
import imageio.v2 as imageio
import os

colors = ["#e0f7fa", "#b2ebf2", "#80deea", "#4dd0e1", "#26c6da", 
          "#00bcd4", "#00acc1", "#0097a7", "#00838f", "#006064",
          "#004f5c", "#003a4a", "#002938", "#001921", "#000c12"]
custom_cmap = LinearSegmentedColormap.from_list("custom_blue_15", colors, N=15)
dir_data = xr.open_dataset('../../data/th.nc')
speed_data = xr.open_dataset('../../data/vs.nc')
lon = dir_data['lon'].values
lat = dir_data['lat'].values
num_arrows = 1000
num_frames = 10
step_lon, step_lat = len(lon) // int(np.sqrt(num_arrows)), len(lat) // int(np.sqrt(num_arrows))
lon_indices = np.arange(0, len(lon), step_lon)[:int(np.sqrt(num_arrows))]
lat_indices = np.arange(0, len(lat), step_lat)[:int(np.sqrt(num_arrows))]
sample_lon, sample_lat = np.meshgrid(lon[lon_indices], lat[lat_indices])
norm = Normalize(vmin=0, vmax=10)
print("Running...")

def generate_images():
    os.makedirs('../images/color_quiver', exist_ok=True)
    for day in range(0, 92, 92//num_frames):
        wind_speed = speed_data['wind_speed'].isel(day=day).values
        wind_dir = dir_data['wind_from_direction'].isel(day=day).values
        wind_rad = np.deg2rad(wind_dir)
        U = np.sin(wind_rad)
        V = np.cos(wind_rad)
        U_sample = U[lat_indices[:, None], lon_indices]
        V_sample = V[lat_indices[:, None], lon_indices]
        speed_sample = wind_speed[lat_indices[:, None], lon_indices]
        fig, ax = plt.subplots(figsize=(10, 7))
        m = Basemap(llcrnrlon=-123, llcrnrlat=20, urcrnrlon=-62, urcrnrlat=50,
                    projection='lcc', lat_1=33, lat_2=45, lat_0=39.5, lon_0=-98, ax=ax)
        m.drawmapboundary(fill_color='#A6CAE0')
        m.fillcontinents(color='#E1DCBD', lake_color='#A6CAE0', alpha=0.7)
        m.drawcoastlines(color='#404040', linewidth=0.8)
        m.drawcountries(color='#404040', linewidth=0.6)
        m.drawparallels(np.arange(20,51,10), labels=[1,0,0,0], fontsize=8, color='#808080', linewidth=0.5)
        m.drawmeridians(np.arange(-120,-60,10), labels=[0,0,0,1], fontsize=8, color='#808080', linewidth=0.5)
        x, y = m(sample_lon, sample_lat)
        q = m.quiver(x, y, U_sample, V_sample, speed_sample, cmap=custom_cmap, norm=norm, scale=40)
        plt.colorbar(q, label='Wind Speed (m/s)', orientation='vertical', fraction=0.046, aspect=10)
        plt.title(f"Wind Direction Analysis - Day {day}", fontsize=14, pad=20, fontweight='bold')
        plt.tight_layout()
        fig.savefig(f'../images/color_quiver/day_{day:02d}.png', format='png')
        plt.close(fig)

def create_gif():
    images = []
    for day in range(0, 92, 92 // num_frames):
        images.append(imageio.imread(f'../images/color_quiver/day_{day:02d}.png'))
    imageio.mimsave(f'../gifs/color_quiver_{1 + num_frames}.gif', images, fps=2)

generate_images()
create_gif()
print("Completed")

Running...
Completed
